In [6]:
from pickle import FALSE
import random
import numpy as np
import wandb
import time
import threading #GM AE 152
import torch
from torch.utils.data import Dataset
from pynput import keyboard
from functools import partial
from collections import deque


class KeyboardObserver:
    def __init__(self):
        self.last_value_gripper = None 
        self.reset()
        self.hotkeys = keyboard.GlobalHotKeys(
            {
                "g": partial(self.set_label, 1),  # good
                "b": partial(self.set_label, 0),  # bad
                "c": partial(self.set_gripper, -0.9),  # close
                "v": partial(self.set_gripper, 0.9),  # open
                "f": partial(self.set_gripper, None),  # gripper free
                "x": self.reset_episode,
                "n": self.press_key_new_episode, # neu um die Episode fuer reale Exp. neu zu starten; ANPASSUNG gmeiner 07.02
                "m": self.episode_end_reached,  # neu um die Episode manuell zu beenden; ANPASSUNG gmeiner 07.02
                "y": self.reset_move_client,
                "t": self.evaluate_success,
                "z": self.evaluate_failure,
            }
        )
        self.hotkeys.start()
        self.direction = np.array([0, 0, 0, 0, 0, 0])
        self.listener = keyboard.Listener(
            on_press=self.set_direction, on_release=self.reset_direction
        )
        self.key_mapping = {
            "d": (1, 1),   # left(0, 0.9, 0) | Roboter Welkoordinaten Y-
            "a": (1, -1),  # right(0, -0.9, 0) | Roboter Welkoordinaten Y+
            "s": (0, 1),  # backward (0.9, 0, 0) | Roboter Welkoordinaten X+
            "w": (0, -1),  # forward (-0.9, 0, 0) | Roboter Welkoordinaten X-
            "e": (2, 1),  # down (0,0,0.9) oben | Roboter Welkoordinaten Z+
            "q": (2, -1),  # up (0,0,-0,9) unten | Roboter Welkoordinaten Z-
            "j": (3, -1),  # look left | Roboter Welkoordinaten B - 
            "l": (3, 1),  # look right | Roboter Welkoordinaten B +
            "k": (4, -1),  # look up | Roboter Welkoordinaten C-
            "i": (4, 1),  # look down | Roboter Welkoordinaten C+
            "o": (5, -1),  # rotate left | Roboter Welkoordinaten A+
            "u": (5, 1),  # rotate right | Roboter Welkoordinaten A-

            #"s": (1, 1),   # left #fuer die richtige Orientierung fuer unseren Demonstrator muss das [s] werden, zuvor a ********s
            #"w": (1, -1),  # right #fuer die richtige Orientierung fuer unseren Demonstrator muss das [w] werden, zuvor d 
            #"a": (0, 1),  # backward #fuer die richtige Orientierung fuer unseren Demonstrator muss das [a] werden, zuvor s ********a
            #"d": (0, -1),  # forward #fuer die richtige Orientierung fuer unseren Demonstrator muss das [d] werden, zuvor w
            #"q": (2, 1),  # down
            #"e": (2, -1),  # up #down? E+A & E+S funktioniert nicht in kombination
            #"j": (3, -1),  # look left #fuer die richtige Orientierung fuer unseren Demonstrator muss das [i] werden, zuvor j 
            #"l": (3, 1),  # look right #fuer die richtige Orientierung fuer unseren Demonstrator muss das [k] werden, zuvor l
            #"k": (4, -1),  # look up #fuer die richtige Orientierung fuer unseren Demonstrator muss das [j] werden, zuvor i
            #"i": (4, 1),  # look down #fuer die richtige Orientierung fuer unseren Demonstrator muss das [l] werden, zuvor k
            #"o": (5, -1),  # rotate left
            #"u": (5, 1),  # rotate right
        }
        self.listener.start()
        return

    def set_label(self, value):
        self.label = value
        print("label set to: ", value)
        return

    def get_label(self):
        return self.label

    def set_gripper(self, value):
        self.gripper_open = value
        print("gripper set to: ", value)
        return

    def get_gripper(self):
        return self.gripper_open

    def set_direction(self, key):
        print("SET direction key:", key, self.key_mapping[key.char])
        try:
            idx, value = self.key_mapping[key.char]
            self.direction[idx] = value
        except (KeyError, AttributeError):
            #print("Fehler bei set_direction")
            pass
        return

    def reset_direction(self, key):
        print("RESET direction key:", key)
        try:
            idx, _ = self.key_mapping[key.char]
            self.direction[idx] = 0
        except (KeyError, AttributeError):
            #print("Fehler bei set_direction")
            pass
        return

    def has_joints_cor(self):
        return self.direction.any()

    def has_gripper_update(self):        
        if self.get_gripper() == self.last_value_gripper:  #      
            response = False

        else:       
            response = True    
        self.last_value_gripper = self.get_gripper()
        return response
        

    def get_ee_action(self):
        return self.direction * 0.9

    def has_gripper_update_feedbacktrain(self):
        return self.get_gripper() is not None

    def reset_episode(self):
        print("Used episode resetting button (reject record)")
        self.reset_button = True
        return


    def begin_new_episode(self): # ANPASSUNG Gmeiner 07.02
        result_available.set()
        return

    def press_key_new_episode(self):
        if self.new_episode_button == False:
            self.new_episode_button = True
            try:
                self.thread.start()
            except (KeyError, AttributeError):
                print("Error bei press_key_new_Episode")
        return

            
        
    #def press_key_new_episode(self): # ANPASSUNG Gmeiner 07.02
    #    self.new_episode_button = True
    #    try:
    #    	self.thread.start()
    #    except (KeyError, AttributeError):
    #    	print("Error bei press_key_new_episode")
    #    return

    def wait_new_episode(self): # ANPASSUNG Gmeiner 07.02
        print("new episode 1")
        self.thread = threading.Thread(target=self.begin_new_episode)
        print(self.thread)
        print("new episode 2")
        result_available.wait(5)
        print("new episode 3")
        result_available.clear()
        print("new episode 4")
        print("resetbutton", self.reset_button)
        print("episode_reached_button", self.episode_reached_button)
        print("new_episode_button", self.new_episode_button)
        print("reset_move_client_button", self.reset_move_client_button)
        self.reset()
        print("reset conducted")
        print("resetbutton DANACH", self.reset_button)
        print("episode_reached_button DANACH", self.episode_reached_button)
        print("new_episode_button DANACH", self.new_episode_button)
        print("reset_move_client_button DANACH", self.reset_move_client_button)
        return
    
    def episode_end_reached(self): # ANPASSUNG Gmeiner 07.02
        print("Used episode ending button")
        self.episode_reached_button = True
        return 

    def evaluate_success(self):
        print("Reports manipulation success")
        self.success_button = True
        return

    def evaluate_failure(self):
        print("Report manipulation failure")
        self.failure_button = True

    def reset(self):
        self.set_label(1)
        self.set_gripper(0.9) #gripper open ->0.9 zuvor none
        self.reset_button = False
        self.episode_reached_button = False # Gmeiner
        self.new_episode_button = False # Gmeiner
        self.reset_move_client_button = False
        self.success_button = False
        self.failure_button = False
        return

    def reset_move_client(self): # ANPASSUNG Gmeiner 07.02
        print("Used move client resetting button")
        self.reset_move_client_button = True
        return 


class MetricsLogger:
    def __init__(self):
        self.total_successes = 0
        self.total_episodes = 0
        self.total_steps = 0
        self.total_cor_steps = 0
        self.total_pos_steps = 0
        self.total_neg_steps = 0
        self.episode_metrics = deque(maxlen=1)
        self.reset_episode()
        return

    def reset_episode(self):
        self.episode_reward = 0
        self.episode_steps = 0
        self.episode_cor_steps = 0
        self.episode_pos_steps = 0
        self.episode_neg_steps = 0
        return

    def log_step(self, reward, feedback):
        self.episode_reward += reward
        self.episode_steps += 1
        if feedback == -1:
            self.episode_cor_steps += 1
        elif feedback == 1:
            self.episode_pos_steps += 1
        elif feedback == 0:
            self.episode_neg_steps += 1
        else:
            raise NotImplementedError
        return

    def log_episode(self, current_episode):
        episode_metrics = {
            "reward": self.episode_reward,
            "ep_cor_rate": self.episode_cor_steps / self.episode_steps,
            "ep_pos_rate": self.episode_pos_steps / self.episode_steps,
            "ep_neg_rate": self.episode_neg_steps / self.episode_steps,
            "episode": current_episode,
        }
        self.append(episode_metrics)
        self.total_episodes += 1
        if self.episode_reward > 0:
            self.total_successes += 1
        self.total_steps += self.episode_steps
        self.total_cor_steps += self.episode_cor_steps
        self.total_pos_steps += self.episode_pos_steps
        self.total_neg_steps += self.episode_neg_steps
        self.reset_episode()
        return

    def log_session(self):
        success_rate = self.total_successes / self.total_episodes
        cor_rate = self.total_cor_steps / self.total_steps
        pos_rate = self.total_pos_steps / self.total_steps
        neg_rate = self.total_neg_steps / self.total_steps
        wandb.run.summary["success_rate"] = success_rate
        wandb.run.summary["total_cor_rate"] = cor_rate
        wandb.run.summary["total_pos_rate"] = pos_rate
        wandb.run.summary["total_neg_rate"] = neg_rate
        return

    def append(self, episode_metrics):
        self.episode_metrics.append(episode_metrics)
        return

    def pop(self):
        return self.episode_metrics.popleft()

    def empty(self):
        return len(self.episode_metrics) == 0

In [7]:
kobs = KeyboardObserver()
print(5)

label set to:  1
gripper set to:  0.9
5
SET direction key: 's' (0, 1)
RESET direction key: 's'
SET direction key: 'a' (1, -1)
RESET direction key: 'a'
SET direction key: 'd' (1, 1)
RESET direction key: 'd'


Unhandled exception in listener callback
Traceback (most recent call last):
  File "/home/faps/anaconda3/envs/ceiling_env/lib/python3.8/site-packages/pynput/_util/__init__.py", line 211, in inner
    return f(self, *args, **kwargs)
  File "/home/faps/anaconda3/envs/ceiling_env/lib/python3.8/site-packages/pynput/_util/xorg.py", line 474, in _handler
    self._handle(self._display_stop, event)
  File "/home/faps/anaconda3/envs/ceiling_env/lib/python3.8/site-packages/pynput/keyboard/_xorg.py", line 564, in _handle
    self.on_press(key)
  File "/home/faps/anaconda3/envs/ceiling_env/lib/python3.8/site-packages/pynput/_util/__init__.py", line 127, in inner
    if f(*args) is False:
  File "/tmp/ipykernel_98771/2612605630.py", line 85, in set_direction
    print("SET direction key:", key, self.key_mapping[key.char])
AttributeError: 'Key' object has no attribute 'char'


Used move client resetting button
Used move client resetting button
Used move client resetting button
gripper set to:  0.9
gripper set to:  0.9
gripper set to:  0.9
gripper set to: gripper set to:  -0.9
gripper set to:  -0.9
 -0.9
gripper set to:  0.9
gripper set to:  0.9
gripper set to:  0.9
Used move client resetting button
Used move client resetting button
Used move client resetting button
gripper set to:  0.9
gripper set to:  0.9
gripper set to:  0.9
gripper set to:  None
gripper set to:  None
gripper set to:  None
label set to:  0
label set to:  0
label set to:  0
gripper set to:  -0.9
gripper set to:  -0.9
gripper set to:  -0.9
Reports manipulation success
Reports manipulation success
Reports manipulation success
Used episode resetting button (reject record)Used episode resetting button (reject record)
Used episode resetting button (reject record)

gripper set to:  -0.9
gripper set to:  -0.9
gripper set to:  -0.9
gripper set to:  -0.9
gripper set to:  -0.9
gripper set to:  -0.9
g